# Exploration of factors affecting transformer model performance on a very simple sequence prediction task

In this investigation, I will look at what happens to the rate at which the loss diminishes and a very simple visual validation of the mode. The factors I will be changing are as follows:

## Changing the vocab size:

I will reduce the vocabulary size. I predict a worse performance from the model as tokens will be larger and the model may find it harder to generalise. If the token library gets small enough, the model may resort to using entire strings as tokens. 
These are the vocab sizes I will iterate through, the rate of loss improvement and a qualitative validation base on written out sequences.

| Vocab size | Rate of improvement                                                                 | Performance evaluation |
|------------|-------------------------------------------------------------------------------------|------------------------|
| 5000       | 8.23 start, below 1 after epoch 2, below 0.1 after epoch 5                        | Consistently correct   |
| 1000       | 8.16, 0.5 then straight to 0.06 which is lower than 5000 tokens ever managed      | Consistently correct   |
| 500        | Longer training time, 5.89 after 1, 0.20 then drops to 0.07 and then 0.05 by epoch 10 – the lowest that has been managed so far | Consistently correct   |
| 100        | Very quick improvement in performance – 3.88 to 0.06 and reaching 0.45 by epoch 10 | Consistently correct   |
| 50         | 3.5 to start. Drops to 0.03 immediately – fastest training speed so far. 0.032 by epoch 10 | Consistently correct   |
| 20         | 2.9 to start, error drops to 0.042 then increases slightly. By epoch 10 it is 0.038 | Consistently correct   |
| 15         | 3.2 to start, then 0.044, increasing to 0.048 and dropping back down to 0.040 by epoch 10 | Consistently correct   |
| 14         | 2.7 to start, drops to 0.39, increases to 0.045 then decreases again to 0.037 by epoch 10 | Consistently correct   |
| Below 14   | Code reaches an error                                                               | -                      |

Not surprisingly, a vocab of 5000 was excessive for this task and performance improved as the vocabulary went down. Performance increased the fastest with a vocab of 50 so for the remainder of testing, the vocab will be set there. It is also interesting to see the error increase and then decrease again. Likely moving out of a local minimum. 

## Positional Encoding

Positional encoding is essential because it helps the model gain an idea of how the location of a token affects the result. There is little positional context required for the sequences which are being generated here so seeing if removing the positional encoding has any impact will be interesting to see. 

| Positional Encoding | Rate of improvement                                                                 | Performance evaluation                                                                  |
|---------------------|-------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------|
| Yes                 | 3.5 to start. Drops to 0.033 immediately. 0.032 by epoch 10                        | Consistently correct                                                                      |
| No                  | 4.7 start, to 0.06 then 0.05, 0.049 by epoch 7 and 0.04 by epoch 10                | Consistently correct, likely due to the fact it is not predicting long sequences |

Interestingly, it seems that without positional encoding the learning improves slower compared with the first training with a vocab of 50 and positional encoding. However, when retraining the model with a vocab of 50 and positional encoding, the training rate is similar to the one without positional encoding. This suggests positional encoding for this problem doesn’t matter as much.

## Number of attention heads

The number of heads is used to speed up the attention mechanism. It splits the dimensions of the model by the number of heads and conducts the matrix multiplications in parallel. This means that for low numbers of heads training should be lower. A high number of heads can also either lead to over fitting or ideal performance depending on the dataset size and model dimensions. 

| Number of attention heads | Rate of improvement                                                                                                                               | Performance evaluation |
|---------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------|------------------------|
| 128                       | Training is significantly slower, and after 10 epochs the model reaches a performance of 0.06                                                     | Consistently correct   |
| 32                        | 4.05 to start, then hovers around 0.05 until epoch 7 and reach 0.045 by epoch 10                                                                  | Consistently correct   |
| 16                        | 3.8 to start, dropping to 0.056. down then up then down to 0.04 at epoch 10                                                                         | Consistently correct   |
| 4                         | Longer training time, 5.89 after 1, 0.20 then drops to 0.07 and then 0.05 by epoch 10 – the lowest that has been managed so far                    | Consistently correct   |
| 1                         | comparable                                                                                                                                        | Consistently correct   |

The training was slowest when the number of heads was high and did not change significantly otherwise. The increase in the training time could be due to the increased data movement and memory usage of the large number of heads. Additionally, the slower increase in performance could be because there is less generalisation over the whole input – many heads make global trends harder to identify. 

## Dropout

Dropout is a regularisation technique used to prevent overfitting. Theoretically it should slow the learning rate as the fraction of nodes in each layer being ‘forgotten’ increases. 

| Dropout | Rate of Improvement                                                                                             | Performance Evaluation |
|---------|-----------------------------------------------------------------------------------------------------------------|------------------------|
| 0.1     | Baseline training speed. Works quickly and loss is reduced quickly.                                            | consistently correct   |
| 0.5     | Slower decrease in loss at the beginning of training but reaches a similar performance by epoch 10 as other models. | consistently correct   |
| 0.8     | Much slower learning rate as expected                                                                          | consistently correct   |
| 0.99    | The slowest by far, did not get below a loss of 3                                                               | consistently correct   |

As expected, training with a dropout of 0.99 was slowest to train because 99% of the information learnt in each layer is not being carried over to the next one. 

## Layers

The number of layers links to the complexity of the task trying to be modelled. More layers allow higher order or longer range connections to be found. For this task, it is likely that fewer layers will be needed because the relationships being modelled are not complex.

| Number of Layers | Rate of Improvement                                                                                                                                                                                             | Performance Evaluation |
|------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------|
| 2                | Learning is much faster and for the amount of time spent training has been the best so far – I wonder what the relationship is between layers and performance – if you have simpler data is it better to have fewer layers because I have really simple data here and changing the layers has been phenomenal | consistently correct   |
| 4                | Not even comparable to 2 layers. Very interesting to see                                                                                                                                                           | consistently correct   |
| 6                | Standard, takes longer to train than 2 and 4 layers                                                                                                                                                               | consistently correct   |
| 10               | Much longer for each epoch, not nearly as efficient as 2 layers, comparable performance to 4                                                                                                                        | consistently correct   |

The best performance is achieved with 2 layers which was expected. Somewhat un-expected is that the actual rate of learning by epochs was faster with 2 layers too which infers that the higher layer count models are hindered by having to update gradients in all those layers, leading to slower learning at each epoch. 

## Model dimensions

The model dimension is the length of your embedding vector. Each vector is assigned to a token in your vocabulary. The longer each vector is, the more information can be stored in each one which means they can handle more complex relationships. However bigger is not always better and a larger d_model can significantly increase training time. 

| Dimension | Description | Correctness |
|-----------|-------------|-------------|
| 64        | Much faster to iterate through training epochs - a world of difference but about 10x worse in terms of performance, lowest reached is 0.47 (not 0.047) | Consistently correct |
| 256       | About double the speed of 512, understandably but with similar performance interestingly | Consistently correct |
| 512       | This is the standard dimension of the model in the tutorial | Consistently correct |
| 1024      | Training takes about double the time which is expected. It seemingly performs worse. Once it finds a slope it follows it and there is more up and down with finding local minima and then getting out of them. | Consistently correct |

It is clear to see both the difference in performance and training time. A d_model of 64 had poor performance but trained incredibly quickly and a d_model of 1024 was a lot slower, seemingly performance worse. 
After Manually optimising, layers = 2, d_model = 256 and vocab = 50 seems to be the best parameters. 
